<a href="https://colab.research.google.com/github/Siddharth-Singh-2004/Combined_CU-BEMS_Transformer/blob/main/Combined_CU_BEMS_Transformer_with_RH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math

In [151]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [152]:
df=pd.read_csv('/content/drive/MyDrive/combined_rh.csv')

In [153]:
df['Date'] = pd.to_datetime(df['Date'])

In [154]:
df

,Date,total_power,z1_S1(RH%)_2,z2_S1(RH%)_2,z3_S1(RH%)_2,z4_S1(RH%)_2,z1_S1(RH%)_3,z2_S1(RH%)_3,z4_S1(RH%)_3,z5_S1(RH%)_3,...,z4_S1(RH%)_5,z5_S1(RH%)_5,z1_S1(RH%)_6,z2_S1(RH%)_6,z4_S1(RH%)_6,z5_S1(RH%)_6,z1_S1(RH%)_7,z2_S1(RH%)_7,z4_S1(RH%)_7,z5_S1(RH%)_7
0,2018-07-01 00:00:00,132184.91,58.93,64.89,68.22,0.0,60.67,63.37,65.66,71.16,...,70.36,68.51,72.19,70.56,71.76,68.48,64.42,66.24,65.16,66.72
1,2018-07-01 01:00:00,134281.82,58.91,64.89,68.21,0.0,60.67,63.37,65.65,71.16,...,70.37,68.50,72.18,70.56,71.76,68.48,64.42,66.26,65.17,66.70
2,2018-07-01 02:00:00,133303.13,58.94,64.89,68.24,0.0,60.67,63.37,65.63,71.18,...,70.37,68.48,72.18,70.56,71.76,68.50,64.42,66.28,65.18,66.70
3,2018-07-01 03:00:00,136726.49,58.94,64.89,68.28,0.0,60.66,63.36,0.00,71.19,...,70.38,68.48,72.17,70.57,71.76,68.51,64.42,66.29,65.18,66.70
4,2018-07-01 04:00:00,140079.97,58.94,64.90,68.30,0.0,60.66,63.36,0.00,71.21,...,70.38,68.46,72.17,70.57,71.76,68.52,64.44,66.32,65.18,66.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,2018-12-31 19:00:00,7040.24,54.10,62.63,62.78,0.0,59.40,62.21,64.44,69.57,...,65.90,65.52,61.31,62.77,63.86,63.36,62.55,0.00,62.70,63.35
4412,2018-12-31 20:00:00,6937.30,53.98,62.62,62.72,0.0,59.50,62.23,64.42,69.57,...,65.80,65.52,61.31,62.77,63.86,63.36,62.55,0.00,62.69,63.37
4413,2018-12-31 21:00:00,6498.20,54.10,62.59,62.46,0.0,59.50,62.24,64.42,69.57,...,65.70,65.51,61.32,62.78,63.85,63.35,62.55,0.00,62.64,63.38
4414,2018-12-31 22:00:00,6233.01,54.20,62.58,62.37,0.0,59.60,62.24,64.41,69.57,...,65.80,65.51,61.33,62.78,63.86,63.33,62.55,0.00,62.63,63.39


In [155]:
# Function to process each group
def process_group(group):
    summed_total_power = group['total_power'].sum()
    averaged_columns = group.drop(columns=['total_power', 'Date']).mean()
    date = group['Date'].iloc[0]  # Keep the first date of the group or handle dates as needed
    return pd.Series([date, summed_total_power, *averaged_columns], index=['Date', 'total_power', *averaged_columns.index])

# Create the new dataframe by applying the function to each group of 24 rows
new_df = df.groupby(df.index // 24).apply(process_group).reset_index(drop=True)

new_df

,Date,total_power,z1_S1(RH%)_2,z2_S1(RH%)_2,z3_S1(RH%)_2,z4_S1(RH%)_2,z1_S1(RH%)_3,z2_S1(RH%)_3,z4_S1(RH%)_3,z5_S1(RH%)_3,...,z4_S1(RH%)_5,z5_S1(RH%)_5,z1_S1(RH%)_6,z2_S1(RH%)_6,z4_S1(RH%)_6,z5_S1(RH%)_6,z1_S1(RH%)_7,z2_S1(RH%)_7,z4_S1(RH%)_7,z5_S1(RH%)_7
0,2018-07-01,3331648.08,59.130000,64.835833,62.535000,0.0,50.505833,63.345000,60.087500,71.199583,...,70.350833,68.412500,72.298750,64.697083,71.754583,62.775417,64.512917,66.514583,65.261250,66.695000
1,2018-07-02,4106154.33,44.784167,64.737500,67.584583,0.0,49.988750,63.306667,65.511250,70.953750,...,64.480833,68.332917,72.096667,70.566250,71.679167,68.446667,64.524167,66.983333,59.944583,66.791250
2,2018-07-03,3933121.37,49.756250,64.667917,67.627917,0.0,59.954583,63.324583,65.479583,70.852917,...,64.515417,68.240000,71.910833,70.575000,71.640417,68.428333,53.959167,67.269167,54.569167,66.911667
3,2018-07-04,3952341.37,59.470833,64.601667,68.033333,0.0,60.028333,63.317083,65.472083,65.332083,...,70.345833,56.804583,71.799167,70.540000,71.650833,68.637917,65.284583,67.429583,65.797500,66.959583
4,2018-07-05,3953446.56,59.502917,64.537500,68.350000,0.0,60.272500,63.308333,65.429583,71.213333,...,67.455833,62.462917,71.708333,64.605000,71.631250,68.711250,65.515833,67.555417,66.263333,67.206250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2018-12-27,381442.85,53.807917,62.784167,62.259583,0.0,49.520417,61.165000,64.600000,68.870000,...,58.981250,65.197083,60.891667,56.953333,63.535417,62.863750,62.370833,67.297500,61.842083,62.444583
180,2018-12-28,371937.07,51.438750,62.732917,62.267917,0.0,52.640833,61.361250,64.326667,69.205417,...,58.844167,65.217083,61.186250,62.260833,63.568750,57.763750,62.455000,66.885833,62.299167,62.785833
181,2018-12-29,183337.20,42.728750,57.459583,62.782500,0.0,58.409167,61.632917,64.307917,69.318750,...,64.447500,65.384583,61.392500,57.135833,63.665000,63.265833,57.295000,13.932500,62.385417,63.223333
182,2018-12-30,164142.85,53.839167,62.661250,57.314167,0.0,53.875417,56.777500,53.657083,69.547500,...,62.103333,65.480833,61.190833,62.559583,63.756250,57.920417,57.341250,0.000000,62.562083,63.229583


In [156]:
combined = np.array(new_df['total_power'])

In [157]:
max_min = {}

In [158]:
for column in df.columns[1:]:
  max_min[column] = [df[column].max(), df[column].min()]
max_min

{'total_power': [225736.92, 0.0],
 'z1_S1(RH%)_2': [63.5, 0.0],
 'z2_S1(RH%)_2': [75.9, 0.0],
 'z3_S1(RH%)_2': [72.9, 0.0],
 'z4_S1(RH%)_2': [0.0, 0.0],
 'z1_S1(RH%)_3': [65.39, 0.0],
 'z2_S1(RH%)_3': [66.69, 0.0],
 'z4_S1(RH%)_3': [70.9, 0.0],
 'z5_S1(RH%)_3': [75.8, 0.0],
 'z1_S1(RH%)_4': [67.38, 0.0],
 'z2_S1(RH%)_4': [72.83, 0.0],
 'z4_S1(RH%)_4': [65.86, 0.0],
 'z5_S1(RH%)_4': [66.9, 0.0],
 'z1_S1(RH%)_5': [79.37, 0.0],
 'z2_S1(RH%)_5': [70.9, 0.0],
 'z4_S1(RH%)_5': [72.8, 0.0],
 'z5_S1(RH%)_5': [70.7, 0.0],
 'z1_S1(RH%)_6': [72.9, 0.0],
 'z2_S1(RH%)_6': [72.8, 0.0],
 'z4_S1(RH%)_6': [71.95, 0.0],
 'z5_S1(RH%)_6': [70.8, 0.0],
 'z1_S1(RH%)_7': [68.85, 0.0],
 'z2_S1(RH%)_7': [74.5, 0.0],
 'z4_S1(RH%)_7': [71.62, 0.0],
 'z5_S1(RH%)_7': [69.9, 0.0]}

In [159]:
# Normalizing
for column in df.columns[1:]:
  df[column] = (df[column] - max_min[column][1]) / (max_min[column][0] - max_min[column][1])

In [160]:
for column in max_min:
  if max_min[column][0] == 0:
    df = df.drop(columns=[column])

In [161]:
df_without_date = df.drop(columns=['Date'])
array_of_rows = df_without_date.values.tolist()

In [162]:
# Normalizing combined_df
data_max = new_df['total_power'].max()
data_min = new_df['total_power'].min()
new_df['total_power'] = (new_df['total_power'] - data_min) / (data_max - data_min)

In [163]:
new_df

,Date,total_power,z1_S1(RH%)_2,z2_S1(RH%)_2,z3_S1(RH%)_2,z4_S1(RH%)_2,z1_S1(RH%)_3,z2_S1(RH%)_3,z4_S1(RH%)_3,z5_S1(RH%)_3,...,z4_S1(RH%)_5,z5_S1(RH%)_5,z1_S1(RH%)_6,z2_S1(RH%)_6,z4_S1(RH%)_6,z5_S1(RH%)_6,z1_S1(RH%)_7,z2_S1(RH%)_7,z4_S1(RH%)_7,z5_S1(RH%)_7
0,2018-07-01,0.800923,59.130000,64.835833,62.535000,0.0,50.505833,63.345000,60.087500,71.199583,...,70.350833,68.412500,72.298750,64.697083,71.754583,62.775417,64.512917,66.514583,65.261250,66.695000
1,2018-07-02,0.994433,44.784167,64.737500,67.584583,0.0,49.988750,63.306667,65.511250,70.953750,...,64.480833,68.332917,72.096667,70.566250,71.679167,68.446667,64.524167,66.983333,59.944583,66.791250
2,2018-07-03,0.951201,49.756250,64.667917,67.627917,0.0,59.954583,63.324583,65.479583,70.852917,...,64.515417,68.240000,71.910833,70.575000,71.640417,68.428333,53.959167,67.269167,54.569167,66.911667
3,2018-07-04,0.956003,59.470833,64.601667,68.033333,0.0,60.028333,63.317083,65.472083,65.332083,...,70.345833,56.804583,71.799167,70.540000,71.650833,68.637917,65.284583,67.429583,65.797500,66.959583
4,2018-07-05,0.956279,59.502917,64.537500,68.350000,0.0,60.272500,63.308333,65.429583,71.213333,...,67.455833,62.462917,71.708333,64.605000,71.631250,68.711250,65.515833,67.555417,66.263333,67.206250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2018-12-27,0.063812,53.807917,62.784167,62.259583,0.0,49.520417,61.165000,64.600000,68.870000,...,58.981250,65.197083,60.891667,56.953333,63.535417,62.863750,62.370833,67.297500,61.842083,62.444583
180,2018-12-28,0.061437,51.438750,62.732917,62.267917,0.0,52.640833,61.361250,64.326667,69.205417,...,58.844167,65.217083,61.186250,62.260833,63.568750,57.763750,62.455000,66.885833,62.299167,62.785833
181,2018-12-29,0.014315,42.728750,57.459583,62.782500,0.0,58.409167,61.632917,64.307917,69.318750,...,64.447500,65.384583,61.392500,57.135833,63.665000,63.265833,57.295000,13.932500,62.385417,63.223333
182,2018-12-30,0.009520,53.839167,62.661250,57.314167,0.0,53.875417,56.777500,53.657083,69.547500,...,62.103333,65.480833,61.190833,62.559583,63.756250,57.920417,57.341250,0.000000,62.562083,63.229583


In [164]:
# Custom dataset
class TimeSeriesDataset(Dataset):

  def __init__(self, training_sequence, target_sequence, sequence_length):
    self.training_sequence = np.asarray(training_sequence, dtype=np.float32)
    self.target_sequence = np.asarray(target_sequence, dtype=np.float32)
    self.sequence_length = sequence_length

  def __len__(self):
    return len(self.target_sequence) - self.sequence_length

  def __getitem__(self, idx):
    x = self.training_sequence[idx*24 : (idx+self.sequence_length)*24]
    y = self.target_sequence[idx+self.sequence_length]
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)



In [165]:
number_of_days = 14
sequence_length = 4
train_size = number_of_days
test_size = 7

In [166]:
training_sequence = array_of_rows[-((train_size + test_size)*24):-(test_size*24)]
target_sequence = new_df['total_power'][-train_size - test_size:-(test_size)].values

In [167]:
dataset = TimeSeriesDataset(training_sequence, target_sequence, sequence_length=sequence_length)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [168]:
for batch_idx, (x, y) in enumerate(dataloader):
  print(f"Batch {batch_idx+1}:")
  print(f"x: {x}")
  print(f"y: {y}")

Batch 1:
x: tensor([[[0.0415, 0.0000, 0.8897,  ..., 0.7957, 0.7914, 0.7288],
         [0.0408, 0.0000, 0.8887,  ..., 0.7960, 0.7903, 0.7272],
         [0.0407, 0.8331, 0.8881,  ..., 0.7962, 0.7879, 0.7263],
         ...,
         [0.0346, 0.8397, 0.8422,  ..., 0.8094, 0.7114, 0.6817],
         [0.0348, 0.8376, 0.8430,  ..., 0.8161, 0.7113, 0.6821],
         [0.0335, 0.8373, 0.8485,  ..., 0.8071, 0.7113, 0.6818]],

        [[0.0410, 0.7546, 0.9020,  ..., 0.7979, 0.8140, 0.7950],
         [0.0411, 0.7537, 0.9047,  ..., 0.7954, 0.8149, 0.7910],
         [0.0406, 0.7528, 0.9022,  ..., 0.7937, 0.8164, 0.7868],
         ...,
         [0.0323, 0.7696, 0.8593,  ..., 0.7915, 0.7203, 0.6898],
         [0.0313, 0.7683, 0.8582,  ..., 0.7987, 0.7199, 0.6900],
         [0.0308, 0.7674, 0.8682,  ..., 0.7936, 0.7194, 0.6897]],

        [[0.0426, 0.8942, 0.8584,  ..., 0.8917, 0.7092, 0.6794],
         [0.0432, 0.8894, 0.8601,  ..., 0.9008, 0.7093, 0.6791],
         [0.0427, 0.8876, 0.8614,  ..., 0.9067

In [169]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [170]:
# Transformer model
class TimeSeriesTransformer(nn.Module):

  def __init__(self, input_size, d_model, n_heads, num_layers):
    super(TimeSeriesTransformer, self).__init__()
    self.embedding = nn.Linear(input_size, d_model)
    self.positional_embedding = nn.Parameter(torch.zeros(300, d_model))
    self.transformer = nn.Transformer(d_model, n_heads, num_layers)
    self.fc_out = nn.Linear(d_model, 1)

  def forward(self, src):
    src = self.embedding(src).to(device)
    src = src + self.positional_embedding[:src.size(1), :]
    src = src.permute(1, 0, 2).to(device)
    transformer_output = self.transformer(src, src).to(device)
    transformer_output = transformer_output.permute(1, 0, 2).to(device)
    output = self.fc_out(transformer_output).to(device)
    return output[:, -1, :]



In [171]:
input_size = len(array_of_rows[0])
d_model = 64
n_heads = 8
num_layers = 2
model = TimeSeriesTransformer(input_size, d_model, n_heads, num_layers).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [179]:
# Training loop
epochs = 10
for epoch in range(epochs):
  epoch_loss = 0.0
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.unsqueeze(-1).to(device)
    output = model(x).to(device)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()

  epoch_loss /= len(dataloader)

  print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

Epoch 1, Loss: 0.0011
Epoch 2, Loss: 0.0021
Epoch 3, Loss: 0.0076
Epoch 4, Loss: 0.0135
Epoch 5, Loss: 0.0162
Epoch 6, Loss: 0.0146
Epoch 7, Loss: 0.0099
Epoch 8, Loss: 0.0048
Epoch 9, Loss: 0.0014
Epoch 10, Loss: 0.0009


In [180]:
forecast_length = 3
true_values = []
predicted_values = []

In [181]:
model.eval()
with torch.no_grad():
  for num in range(1, forecast_length+1):
    print(num)
    last_sequence = torch.tensor(array_of_rows[-((sequence_length)*24) - (num*24):-(num*24)], dtype=torch.float32).unsqueeze(0).to(device)
    normalized_forecast = model(last_sequence)
    forecast = normalized_forecast.item() * (data_max - data_min) + data_min
    predicted_values.append(forecast)
    true_values.append(combined[-(num)])

    print(f'Forecast {num}: {forecast:.4f}')
    print(f'Actual {num}: {combined[-(num)]}')

print(f'True values: {true_values}')
print(f'Predicted values: {predicted_values}')

1
Forecast 1: 196444.9161
Actual 1: 126040.98999999999
2
Forecast 2: 261817.3470
Actual 2: 164142.84999999998
3
Forecast 3: 373575.4176
Actual 3: 183337.2
True values: [126040.98999999999, 164142.84999999998, 183337.2]
Predicted values: [196444.91612700932, 261817.347044013, 373575.4176213072]


In [182]:
true_values = np.array(true_values)
predicted_values = np.array(predicted_values)

In [183]:
# Calculate MAPE
def calculate_mape(true, pred):
    return np.mean(np.abs((true - pred) / true)) * 100

In [184]:
# Calculate RMSE
def calculate_rmse(true, pred):
    return np.sqrt(np.mean((true - pred) ** 2))

In [185]:
# Calculate R-squared
def calculate_r2(true, pred):
    ss_tot = np.sum((true - np.mean(true)) ** 2)
    ss_res = np.sum((true - pred) ** 2)
    return 1 - (ss_res / ss_tot)

In [186]:
# Calculate MAPE for the provided values
mape = calculate_mape(np.array(true_values), np.array(predicted_values))

print(f'MAPE: {mape:.2f}%')

MAPE: 73.04%


In [187]:
# Calculate RMSE for the provided values
rmse = calculate_rmse(true_values, predicted_values)

print(f'RMSE: {rmse:.2f}')

RMSE: 129984.10


In [188]:
# Calculate R-squared for the provided values
r2 = calculate_r2(true_values, predicted_values)

print(f'R-squared: {r2:.2f}')

R-squared: -28.80


In [189]:
total_true = true_values.sum()
total_predicted = predicted_values.sum()
print(f'True:      {total_true}')
print(f'Predicted: {total_predicted:.2f}')

True:      473521.04
Predicted: 831837.68
